# ids projekti, kivaa :---)

tästä alkaa suuri seikkailu. Olkaamme innokaita ja tehokkaita. 

In [224]:
import json
import pandas as pd
import numpy as np
import urllib.request as ur
import requests
from datetime import timedelta, date
from sklearn.model_selection import train_test_split as tts
from tpot import TPOTClassifier

In [176]:

def date_range(n):
    date_list = []

    for single_date in (date(2017,10,16) - timedelta(n) for n in range(n)):
        date_list.append(single_date.strftime('%Y%m%d'))

    return(date_list)

In [299]:

def fetch_game_scoreboards():
    games_lists = []

    auth=('Enzine', 'Enzine')

    url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/scoreboard.json?fordate={date}'
    
    DATE_RANGE = 100

    for i in date_range(DATE_RANGE):
        json = requests.get(url.format(date=i), auth=auth).json()
        
        try:
            json['scoreboard']['gameScore']
        except:
            continue
        
        games_lists.append(json['scoreboard']['gameScore'])

    return(games_lists)

In [248]:
#date_range(30)

In [300]:
game_data = fetch_game_scoreboards()

game_data_list = np.hstack(game_data)

Jos kotijoukkoe voittaa, winner = 1

In [301]:

def parse_game(g):
    data = dict()

    data['away_team_id'] = g['game']['awayTeam']['ID']
    data['home_team_id'] = g['game']['homeTeam']['ID']
    data['home_score'] = g['homeScore']
    data['away_score'] = g['awayScore']
    data['away_shots'] = g['awayShots']
    data['home_shots'] = g['awayShots']
    
    if data['home_score'] > data['away_score']:
        data['winner'] = 1
    else:
        data['winner'] = 0

    return(pd.Series(data))


In [302]:
df = pd.DataFrame()

for g in game_data_list:
    df = df.append(parse_game(g), ignore_index=True)
    
df['winner'] = df['winner'].astype('category')
df

,away_score,away_shots,away_team_id,home_score,home_shots,home_team_id,winner
0,3,26,1,2,26,16,0.0
1,1,22,11,3,22,142,1.0
2,3,31,15,1,31,29,0.0
3,2,25,8,3,25,28,1.0
4,1,29,3,2,29,47,1.0
5,1,29,17,2,29,1,1.0
6,3,19,7,2,19,9,0.0
7,3,46,4,4,46,10,1.0
8,4,22,12,3,22,14,0.0
9,2,23,5,8,23,6,1.0


In [303]:
val = df['winner']
df = df.drop('winner', axis=1)

X_train, X_test, y_train, y_test = tts(df, val, train_size=0.75, test_size=0.25, random_state=7)

print(X_train.shape, y_train.shape, y_train.shape, y_test.shape)

(60, 6) (60,) (60,) (20,)


In [304]:
tpot = TPOTClassifier(generations=10, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)


Optimization Progress:  18%|█▊        | 100/550 [00:21<02:12,  3.40pipeline/s]

Generation 1 - Current best internal CV score: 1.0


Optimization Progress:  27%|██▋       | 150/550 [00:39<03:22,  1.98pipeline/s]

Generation 2 - Current best internal CV score: 1.0


Optimization Progress:  36%|███▋      | 200/550 [01:00<02:11,  2.67pipeline/s]

...

Generation 8 - Current best internal CV score: 1.0


Optimization Progress:  91%|█████████ | 500/550 [04:02<00:27,  1.82pipeline/s]

Generation 9 - Current best internal CV score: 1.0


Generation 10 - Current best internal CV score: 1.0

Best pipeline: LinearSVC(input_matrix, C=5.0, dual=True, loss=hinge, penalty=l2, tol=0.1)


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT....45,
        0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,  0.9 ,
        0.95,  1.  ])}}}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=10, max_eval_time_mins=5,
        max_time_mins=None, mutation_rate=0.9, n_jobs=1, offspring_size=50,
        periodic_checkpoint_folder=None, population_size=50,
        random_state=None, scoring=None, subsample=1.0, verbosity=2,
        warm_start=False)

In [305]:
mod_X_test = X_test

mod_X_test['away_score'] = 0
mod_X_test['home_score'] = 0
mod_X_test['away_shots'] = 0
mod_X_test['home_shots'] = 0


print(tpot.score(mod_X_test, y_test))
#tpot.export('teesticoodi.py')





0.65


/Users/Jussi/vens/tf/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Jussi/vens/tf/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/Jussi/vens/tf/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind